# Table of contents

[1. Introduction](#Introduction)

[2. Obtaining the data](#Obtaining-the-data)

[3. Creating and saving models](#Creating-and-saving-models)

In [1]:
# Importing the required libraries
import pandas as pd
pd.set_option('display.max_columns', 50) # Display up to 50 columns at a time
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import cm
plt.style.use('seaborn')
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12,5
import glob # To read all csv files in the directory
import seaborn as sns
import calendar
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, f1_score, precision_recall_fscore_support
import itertools
import time
import xgboost as xgb
import pickle as pk
from sklearn.externals import joblib

In [10]:
# Loading the datasets
X1 = pd.read_csv('processed_datasets/observations1.csv', index_col=0)
y1 = pd.read_csv('processed_datasets/labels1.csv', index_col=0)
X2 = pd.read_csv('processed_datasets/observations2.csv', index_col=0)
y2 = pd.read_csv('processed_datasets/labels2.csv', index_col=0)
X3 = pd.read_csv('processed_datasets/observations3.csv', index_col=0)
y3 = pd.read_csv('processed_datasets/labels3.csv', index_col=0)

In [3]:
# Functionn to train and save models
def train_and_save_model(model, X, y, model_name):
    """train and save model into folder zerol_level_models"""
    model.fit(X, y)
    filename = 'zero_level_models/{}.txt'.format(model_name)
    pk.dump(model, open(filename, 'wb'))

In [5]:
# Function to create all models
def create_models(models_to_create, datasets):
    for dataset_number, dataset in enumerate(datasets):
        total_count_models = 1
        X, y = dataset
        for model_information in models_to_create:
            model, number_of_models_to_create = model_information
            for i in range(number_of_models_to_create):
                train_and_save_model(model, X, y, 'model{}_{}'.format(str(dataset_number+1), total_count_models))
                total_count_models += 1

## Classifiers

In [6]:
# Logistic Regression
logreg = LogisticRegression(C=0.01, fit_intercept=True, penalty='l1')

# Random Forest
rf = RandomForestClassifier(min_samples_split=0.001, verbose=2, n_estimators=200, max_depth=35)

# XGBoost
xgb = xgb.XGBClassifier(learning_rate=0.1, max_depth=35, min_child_weight=100, n_estimators=200, subsample=0.7)

# AdaBoost
adaboost = AdaBoostClassifier(learning_rate=0.3, n_estimators=300)

In [8]:
X1.head()

,blurb_length,usd_goal,name_length,creation_to_launch_days,campaign_days,category_art,category_comics,category_crafts,category_dance,category_design,category_fashion,category_film & video,category_food,category_games,category_journalism,category_music,category_photography,category_publishing,category_technology,category_theater,country_AT,country_AU,country_BE,country_CA,country_CH,...,deadline_month_September,launch_time_10am-12pm,launch_time_10pm-12am,launch_time_12am-2am,launch_time_12pm-2pm,launch_time_2am-4am,launch_time_2pm-4pm,launch_time_4am-6am,launch_time_4pm-6pm,launch_time_6am-8am,launch_time_6pm-8pm,launch_time_8am-10am,launch_time_8pm-10pm,deadline_time_10am-12pm,deadline_time_10pm-12am,deadline_time_12am-2am,deadline_time_12pm-2pm,deadline_time_2am-4am,deadline_time_2pm-4pm,deadline_time_4am-6am,deadline_time_4pm-6pm,deadline_time_6am-8am,deadline_time_6pm-8pm,deadline_time_8am-10am,deadline_time_8pm-10pm
0,0.599954,-0.863064,-0.369130,0.046185,-0.224741,-0.34594,-0.19773,-0.182523,-0.134647,-0.189923,-0.235358,-0.40577,-0.292264,3.839393,-0.14929,-0.412689,-0.186654,-0.334418,-0.34294,-0.168885,-0.047035,-0.152409,-0.050568,-0.217684,-0.055075,...,3.338081,-0.208348,-0.338527,-0.298293,-0.275938,-0.259576,-0.367145,-0.235875,2.445936,-0.203413,-0.387776,-0.191523,-0.368327,-0.196596,-0.343587,-0.307077,-0.245277,-0.317588,-0.324195,-0.313050,2.662370,-0.24725,-0.36068,-0.180051,-0.352531
1,-0.804977,0.694414,0.596395,1.329281,-0.224741,-0.34594,-0.19773,-0.182523,-0.134647,-0.189923,-0.235358,-0.40577,-0.292264,-0.260458,-0.14929,2.423134,-0.186654,-0.334418,-0.34294,-0.168885,-0.047035,-0.152409,-0.050568,-0.217684,-0.055075,...,-0.299573,-0.208348,-0.338527,-0.298293,-0.275938,-0.259576,-0.367145,4.239530,-0.408841,-0.203413,-0.387776,-0.191523,-0.368327,-0.196596,-0.343587,-0.307077,-0.245277,-0.317588,-0.324195,3.194373,-0.375605,-0.24725,-0.36068,-0.180051,-0.352531
2,0.800659,0.573727,0.826781,-0.134553,2.325957,-0.34594,-0.19773,-0.182523,-0.134647,-0.189923,-0.235358,-0.40577,-0.292264,3.839393,-0.14929,-0.412689,-0.186654,-0.334418,-0.34294,-0.168885,-0.047035,-0.152409,-0.050568,-0.217684,-0.055075,...,-0.299573,-0.208348,-0.338527,-0.298293,-0.275938,-0.259576,-0.367145,-0.235875,-0.408841,-0.203413,2.578810,-0.191523,-0.368327,-0.196596,-0.343587,-0.307077,-0.245277,-0.317588,-0.324195,-0.313050,2.662370,-0.24725,-0.36068,-0.180051,-0.352531
3,1.001363,-0.270900,0.596395,-0.220362,-0.224741,-0.34594,-0.19773,-0.182523,-0.134647,-0.189923,-0.235358,-0.40577,-0.292264,-0.260458,-0.14929,-0.412689,-0.186654,2.990267,-0.34294,-0.168885,-0.047035,-0.152409,-0.050568,-0.217684,-0.055075,...,-0.299573,-0.208348,-0.338527,-0.298293,-0.275938,-0.259576,-0.367145,-0.235875,-0.408841,-0.203413,-0.387776,-0.191523,2.714977,-0.196596,-0.343587,-0.307077,-0.245277,-0.317588,-0.324195,-0.313050,-0.375605,-0.24725,-0.36068,-0.180051,2.836630
4,-0.202864,-0.142564,-0.369130,1.195768,-0.224741,-0.34594,-0.19773,-0.182523,-0.134647,-0.189923,-0.235358,-0.40577,-0.292264,-0.260458,-0.14929,2.423134,-0.186654,-0.334418,-0.34294,-0.168885,-0.047035,-0.152409,-0.050568,-0.217684,-0.055075,...,-0.299573,-0.208348,-0.338527,-0.298293,-0.275938,3.852439,-0.367145,-0.235875,-0.408841,-0.203413,-0.387776,-0.191523,-0.368327,-0.196596,-0.343587,-0.307077,-0.245277,3.148737,-0.324195,-0.313050,-0.375605,-0.24725,-0.36068,-0.180051,-0.352531


In [9]:
y1.head()

,0
0,1
1,1
2,0
3,1
4,1


In [10]:
adaboost.fit(X1, y1)
y_pred = adaboost.predict(X2)
print(classification_report(y2, y_pred))

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


              precision    recall  f1-score   support

           0       0.76      0.60      0.67     24881
           1       0.73      0.85      0.78     31445

   micro avg       0.74      0.74      0.74     56326
   macro avg       0.74      0.72      0.73     56326
weighted avg       0.74      0.74      0.73     56326



In [13]:
# load the model from disk
loaded_model = joblib.load('zero_level_models/model1_1.txt')
result = loaded_model.score(X2, y2)
print(result)

0.7305862301601392


In [11]:
models_to_create = [(logreg, 5), (rf, 1), (xgb, 5), (adaboost, 5)]
datasets = [(X1, y1), (X2, y2)]

In [8]:
create_models(models_to_create, datasets)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape o

building tree 1 of 200
building tree 2 of 200
building tree 3 of 200
building tree 4 of 200
building tree 5 of 200
building tree 6 of 200
building tree 7 of 200
building tree 8 of 200
building tree 9 of 200
building tree 10 of 200
building tree 11 of 200
building tree 12 of 200
building tree 13 of 200
building tree 14 of 200
building tree 15 of 200
building tree 16 of 200
building tree 17 of 200
building tree 18 of 200
building tree 19 of 200
building tree 20 of 200
building tree 21 of 200
building tree 22 of 200
building tree 23 of 200
building tree 24 of 200
building tree 25 of 200
building tree 26 of 200
building tree 27 of 200
building tree 28 of 200
building tree 29 of 200
building tree 30 of 200
building tree 31 of 200
building tree 32 of 200
building tree 33 of 200
building tree 34 of 200
building tree 35 of 200
building tree 36 of 200
building tree 37 of 200
building tree 38 of 200
building tree 39 of 200
building tree 40 of 200
building tree 41 of 200
building tree 42 of 200
b

[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:   14.3s finished
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/P

building tree 1 of 200
building tree 2 of 200
building tree 3 of 200
building tree 4 of 200
building tree 5 of 200
building tree 6 of 200
building tree 7 of 200
building tree 8 of 200
building tree 9 of 200
building tree 10 of 200
building tree 11 of 200
building tree 12 of 200
building tree 13 of 200
building tree 14 of 200
building tree 15 of 200
building tree 16 of 200
building tree 17 of 200
building tree 18 of 200
building tree 19 of 200
building tree 20 of 200
building tree 21 of 200
building tree 22 of 200
building tree 23 of 200
building tree 24 of 200
building tree 25 of 200
building tree 26 of 200
building tree 27 of 200
building tree 28 of 200
building tree 29 of 200
building tree 30 of 200
building tree 31 of 200
building tree 32 of 200
building tree 33 of 200
building tree 34 of 200
building tree 35 of 200
building tree 36 of 200
building tree 37 of 200
building tree 38 of 200
building tree 39 of 200
building tree 40 of 200
building tree 41 of 200
building tree 42 of 200
b

[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:   13.6s finished
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/P

In [5]:
# Here are the needed ressources
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.layers import Dropout
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.models import load_model

In [27]:
def train_and_save_neural_net_model(X, y, model_name):
    ohe = OneHotEncoder()
    y = ohe.fit_transform(y)
    # let's retrieve the dimension of the input layer
    n = len(X.columns)
    # let's retrieve the dimension of the output layer
    m = 1
    # Neural network
    model = Sequential()
    model.add(Dropout(0.2, input_shape = (n,)))
    model.add(Dense(1000, activation='relu'))
    model.add(Dense(1000, activation='relu'))
    model.add(Dense(1000, activation='relu'))
    model.add(Dense(500, activation='relu'))
    model.add(Dense(2, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # Training
    history = model.fit(X, y, epochs=10, batch_size=64)
    model.save('zero_level_models/{}'.format(model_name))

In [28]:
# Function to create Neural Networks
def create_neural_networks_models(number_of_models_to_create, datasets):
    for dataset_number, dataset in enumerate(datasets):
        total_count_models = 1
        X, y = dataset
        for i in range(number_of_models_to_create):
                train_and_save_neural_net_model(X, y, 'neural_net_model{}_{}'.format(str(dataset_number+1), total_count_models))
                total_count_models += 1

In [29]:
datasets = [(X1, y1), (X2, y2)]
create_neural_networks_models(5, datasets)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


Epoch 1/10
56326/56326 [==============================] - 33s 579us/step - loss: 0.5626 - acc: 0.7121
Epoch 2/10
56326/56326 [==============================] - 32s 563us/step - loss: 0.5472 - acc: 0.7235
Epoch 3/10
56326/56326 [==============================] - 31s 549us/step - loss: 0.5412 - acc: 0.7277
Epoch 4/10
56326/56326 [==============================] - 30s 535us/step - loss: 0.5365 - acc: 0.7274
Epoch 5/10
56326/56326 [==============================] - 37s 653us/step - loss: 0.5334 - acc: 0.7315
Epoch 6/10
56326/56326 [==============================] - 33s 581us/step - loss: 0.5317 - acc: 0.7312
Epoch 7/10
56326/56326 [==============================] - 32s 560us/step - loss: 0.5277 - acc: 0.7338
Epoch 8/10
56326/56326 [==============================] - 32s 566us/step - loss: 0.5242 - acc: 0.7365
Epoch 9/10
56326/56326 [==============================] - 31s 551us/step - loss: 0.5211 - acc: 0.7373
Epoch 10/10
56326/56326 [==============================] - 32s 566us/step - loss: 

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


Epoch 1/10
56326/56326 [==============================] - 36s 632us/step - loss: 0.5654 - acc: 0.7101
Epoch 2/10
56326/56326 [==============================] - 40s 703us/step - loss: 0.5454 - acc: 0.7240
Epoch 3/10
56326/56326 [==============================] - 43s 770us/step - loss: 0.5387 - acc: 0.7285
Epoch 4/10
56326/56326 [==============================] - 44s 781us/step - loss: 0.5349 - acc: 0.7302
Epoch 5/10
56326/56326 [==============================] - 53s 939us/step - loss: 0.5337 - acc: 0.7293
Epoch 6/10
56326/56326 [==============================] - 45s 791us/step - loss: 0.5288 - acc: 0.7345
Epoch 7/10
56326/56326 [==============================] - 37s 656us/step - loss: 0.5258 - acc: 0.7355
Epoch 8/10
56326/56326 [==============================] - 41s 729us/step - loss: 0.5224 - acc: 0.7362
Epoch 9/10
56326/56326 [==============================] - 44s 787us/step - loss: 0.5216 - acc: 0.7372
Epoch 10/10
56326/56326 [==============================] - 44s 777us/step - loss: 

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


...

Epoch 1/10
56326/56326 [==============================] - 26s 461us/step - loss: 0.5729 - acc: 0.7000
Epoch 2/10
56326/56326 [==============================] - 25s 443us/step - loss: 0.5543 - acc: 0.7139
Epoch 3/10
56326/56326 [==============================] - 25s 437us/step - loss: 0.5481 - acc: 0.7198
Epoch 4/10
56326/56326 [==============================] - 25s 448us/step - loss: 0.5450 - acc: 0.7204
Epoch 5/10
56326/56326 [==============================] - 25s 441us/step - loss: 0.5440 - acc: 0.7208
Epoch 6/10
56326/56326 [==============================] - 25s 450us/step - loss: 0.5401 - acc: 0.7217
Epoch 7/10
56326/56326 [==============================] - 25s 447us/step - loss: 0.5355 - acc: 0.7244
Epoch 8/10
56326/56326 [==============================] - 25s 439us/step - loss: 0.5323 - acc: 0.7302
Epoch 9/10
56326/56326 [==============================] - 25s 440us/step - loss: 0.5306 - acc: 0.7272
Epoch 10/10
56326/56326 [==============================] - 25s 439us/step - loss: 

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


Epoch 1/10
56326/56326 [==============================] - 26s 462us/step - loss: 0.5726 - acc: 0.7006
Epoch 2/10
56326/56326 [==============================] - 26s 453us/step - loss: 0.5542 - acc: 0.7126
Epoch 3/10
56326/56326 [==============================] - 25s 453us/step - loss: 0.5492 - acc: 0.7177
Epoch 4/10
56326/56326 [==============================] - 25s 444us/step - loss: 0.5460 - acc: 0.7193
Epoch 5/10
56326/56326 [==============================] - 25s 446us/step - loss: 0.5431 - acc: 0.7189
Epoch 6/10
56326/56326 [==============================] - 25s 444us/step - loss: 0.5378 - acc: 0.7238
Epoch 7/10
56326/56326 [==============================] - 25s 446us/step - loss: 0.5355 - acc: 0.7235
Epoch 8/10
56326/56326 [==============================] - 25s 449us/step - loss: 0.5352 - acc: 0.7262
Epoch 9/10
56326/56326 [==============================] - 25s 446us/step - loss: 0.5315 - acc: 0.7260
Epoch 10/10
56326/56326 [==============================] - 25s 445us/step - loss: 

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


Epoch 1/10
56326/56326 [==============================] - 26s 465us/step - loss: 0.5719 - acc: 0.7017
Epoch 2/10
56326/56326 [==============================] - 26s 453us/step - loss: 0.5546 - acc: 0.7140
Epoch 3/10
56326/56326 [==============================] - 33s 592us/step - loss: 0.5475 - acc: 0.7195
Epoch 4/10
56326/56326 [==============================] - 29s 515us/step - loss: 0.5447 - acc: 0.7197
Epoch 5/10
56326/56326 [==============================] - 29s 508us/step - loss: 0.5434 - acc: 0.7198
Epoch 6/10
56326/56326 [==============================] - 30s 525us/step - loss: 0.5407 - acc: 0.7227
Epoch 7/10
56326/56326 [==============================] - 29s 517us/step - loss: 0.5355 - acc: 0.7264
Epoch 8/10
56326/56326 [==============================] - 28s 502us/step - loss: 0.5357 - acc: 0.7251
Epoch 9/10
56326/56326 [==============================] - 29s 509us/step - loss: 0.5296 - acc: 0.7277
Epoch 10/10
56326/56326 [==============================] - 29s 519us/step - loss: 

In [12]:
model2 = load_model('zero_level_models/neural_net_model1_1.h5')

ohe = OneHotEncoder()
y2 = ohe.fit_transform(y2)

pred_test = model2.predict(X2)

model2.evaluate(X2,y2)

# Accuracy
# Converting predictions to label
pred = list()
for i in range(len(pred_test)):
    pred.append(np.argmax(pred_test[i]))
#Converting one hot encoded test label to label
test = list()
for i in range(y_test.shape[0]):
    test.append(np.argmax(y_test[i]))


# Accuracy
a = accuracy_score(pred,test)
print('Accuracy is:', a*100)
print(f1_score(test, pred, average="macro"))

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


56326/56326 [==============================] - 7s 130us/step
Accuracy is: 72.99826012853744
0.7137349411536646
